In [ ]:
from fastai.tabular.all import *
from fastai.collab import *
from fastai.data.transforms import *
import torch
import numpy as np

In [ ]:
df = pd.read_csv('User ratings.csv')




In [ ]:
df

,Unnamed: 0,Name,Gender,user,rating,title
0,0,EMMA,Female,0,1.0,Grim Reaper
1,1,EMMA,Female,0,0.0,Monkey Wrench
2,2,EMMA,Female,0,1.0,Espresso Martini
3,3,EMMA,Female,0,0.0,Vesper
4,4,EMMA,Female,0,0.0,Long Island Iced Tea
...,...,...,...,...,...,...
6319,641200,ISAAC,Male,700,1.0,Cream Soda
6320,641201,ISAAC,Male,700,0.0,Jackhammer
6321,641202,ISAAC,Male,700,1.0,Mojito
6322,641203,ISAAC,Male,700,1.0,747


In [ ]:
cocktail_strings = df['title'].unique()

# Create mapping of cocktail strings to integer IDs
cocktail_id_map = {s: i for i, s in enumerate(cocktail_strings)}

# Replace cocktail strings with their IDs in the dataset
df['cocktail'] = df['title'].map(cocktail_id_map)

In [ ]:
df = df[['user','cocktail','rating','title']]

In [ ]:
ratings = df

In [ ]:
ratings

,user,cocktail,rating,title
0,0,0,1.0,Grim Reaper
1,0,1,0.0,Monkey Wrench
2,0,2,1.0,Espresso Martini
3,0,3,0.0,Vesper
4,0,4,0.0,Long Island Iced Tea
...,...,...,...,...
6319,700,406,1.0,Cream Soda
6320,700,403,0.0,Jackhammer
6321,700,390,1.0,Mojito
6322,700,395,1.0,747


In [ ]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)

# Define model 
learn = collab_learner(dls, n_factors=50, y_range=(0, 1.0), loss_func = BCEWithLogitsLossFlat())

In [ ]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.726274,0.716927,00:00
1,0.723322,0.716056,00:00
2,0.710728,0.715472,00:00
3,0.692641,0.715463,00:00
4,0.682407,0.715464,00:00


In [ ]:
# lets mock a menu with 10 positions
items_df = pd.DataFrame({'user': 10,
                      'title': ['Bluebird', 'Boston Sour', 'Irish Spring', 'Zipperhead', 'ACID', 'Spritz', 'Kir Royale', 'Affair', 'Lord and Lady', 'Van Vleet']})  #positions on the menu

# learner to get predictions for the items (the probability of liking the item)
preds, _ = learn.get_preds(dl=dls.test_dl(items_df))

In [ ]:
# Get the index of the item with the highest predicted probability
most_likely_item_idx = np.argmax(preds)



In [ ]:
most_likely_item_idx = most_likely_item_idx.item()

In [ ]:
most_likely_item_title = items_df.loc[most_likely_item_idx, 'title']
print(f"Recommended for you: {most_likely_item_title}")


Recommended for you: ACID


In [ ]:
g = ratings.groupby('title')['rating'].count()
top_cocktails = g.sort_values(ascending=False).index.values[:1000]
top_cocktails[:10]


cocktail_bias = learn.model.bias(top_cocktails, is_item=True)
cocktail_bias.shape


mean_ratings = ratings.groupby('title')['rating'].mean()
cocktail_ratings = [(b, i, mean_ratings.loc[i]) for i,b in zip(top_cocktails,cocktail_bias)]

item0 = lambda o:o[0]
items_titles = items_df['title'].unique()

# Filter cocktail_ratings to only include ratings for items  on the menu
filtered_cocktail_ratings = [(b, i, mean_ratings.loc[i]) for b, i, r in cocktail_ratings if i in items_titles]
list = sorted(filtered_cocktail_ratings, key=lambda o: o[0], reverse=True)[0:1]
print(f"Top drink on the menu: {list[0][1]}")






Top drink on the menu: Irish Spring
